In [1]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath("."))
sys.path.append(SCRIPT_DIR)

In [ ]:
from trainer import ldm
from torchvision import transforms, datasets

In [3]:
args = ldm.create_default_args()

In [4]:
args.model_type = "sde"
args.img_size = 32
args.in_channels = 1
args.batch_size = 64
args.save_freq = 2
args.dataset_path = "./data"
args.beta_start = 0.1
args.beta_end = 1


def dequantize(x, nvals=256):
    """[0, 1] -> [0, nvals] -> add uniform noise -> [0, 1]"""
    noise = x.new().resize_as_(x).uniform_()
    x = x * (nvals - 1) + noise
    x = x / nvals
    return x


transform = transforms.Compose(
    [
        # transforms.RandomResizedCrop(args.img_size, scale=(0.8, 1.0)),
        transforms.Resize((args.img_size, args.img_size)),
        transforms.ToTensor(),
        # transforms.Normalize((0.5,), (0.5,)),
        dequantize,
    ]
)

args.dataset = datasets.MNIST(args.dataset_path, transform=transform, download=True)
# args.checkpoint = "ckpt-81.pt"

In [ ]:
ldm.train(args)